# Imports
Import python libraries

In [1]:
import os

# Set dataset parameters
Choose the attributes to load the dataset.

In [2]:


detectors = ['mtcnn_serfiq']  # ,'dlib', 'opencv', 'mediapipe','mtcnn','mtcnn_serfiq'
embedding_models = ['Facenet512','QMagFace','ArcFace']  # 'QMagFace', "ArcFace","Dlib", 'VGG-Face','FaceVACs', 'Facenet512','ArcFace_normalized'
quality_models = ['ser_fiq','tface'] #  'ser_fiq','tface'


# Set LR parameters
Calibration, image filters, face filters, quality filters...

## Calibration

In [3]:
calibrators = ['logit']  # 'KDE', , 'isotonic','logit'
calibration_db = ["SCFace",'LFW'] # "SCFace",'ForenFace','ENFSI','ForenFace','LFW','SCFace','UTKface','XQLFW',
image_filters = ['gender']   # 'yaw','pitch','roll', E.g. 'pose','gender','emotion'
face_image_filters = ['confusion_score']  # 'confusion_score'
quality_filters = ['quality_group'] # 'quality_group'

#all the filters in one dictionary
filters = {'image': image_filters, 'face_image': face_image_filters,'quality_image': quality_filters}

n_calibration_pairs = 10000



## Validation

In [4]:
enfsi_years = [2015]

# Video parameters

In [5]:
import numpy as np
#quality_dropout = [0.01, 0.02, 0.05, 0.10, 0.20, 0.5, 1]
quality_dropout = list(np.linspace(0.1,1.0,10))

## Other parameters
Rest of the configuration

In [6]:
metrics = 'euclidean_l2'
embedding_model_as_scorer = True

# Scenarios
Different filters to do computations

In [7]:
scenario0 = []
scenario1 = ['gender', 'age']
scenario2 = ['yaw', 'pitch']
scenario3 = ['glasses', 'beard']
scenario4 = ['low_quality']
scenario5 = ['headgear']
scenario6 = ['terhorst_quality_rounded']

# Input and output directories
Where the sql database is and which folder the results are saved.

In [8]:
home = os.path.expanduser("~")

# SQL database path and name
input_dir = os.path.join(home, 'video_resources', 'sql_database')
db_name = 'colab_main_dataset'

# Where results are saved
output_dir = os.path.join(home, 'video_resources', 'output_2')

# Preprocess data
Connect to sql database.

In [9]:
from sql_face.alchemy import get_session
from sql_face.tables import *

2023-05-04 20:07:24.006565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
session = get_session(input_dir, db_name)

# Experiments

All parameters are specified in ExperimentalSetup, that creates a different experiment with all the posible combinations of detectors, embedding_models, calibrators, etc.

In [11]:
from lr_video_face.experiments import ExperimentalSetup

In [12]:
experimental_setup = ExperimentalSetup(detectors, 
                                        embedding_models, 
                                        quality_models,
                                        calibrators, 
                                        calibration_db, 
                                        enfsi_years,
                                        filters,
                                        metrics, 
                                        n_calibration_pairs,
                                        embedding_model_as_scorer,
                                        output_dir,
                                        session,
                                        quality_dropout)

# Results

In [13]:
""" from sql_face.tables import Image, Quality
query = session.query(Image).filter(Image.image_quality == Quality.MEDIUM).all()

a = len(query)
print (a) """


' from sql_face.tables import Image, Quality\nquery = session.query(Image).filter(Image.image_quality == Quality.MEDIUM).all()\n\na = len(query)\nprint (a) '

In [14]:
'''for exp in experimental_setup:
    exp.perform()
    break  '''

'for exp in experimental_setup:\n    exp.perform()\n    break  '

In [15]:
from lr_video_face.evaluators import GlobalEvaluator  

In [16]:
global_results = GlobalEvaluator(experimental_setup)    

100%|██████████| 6/6 [35:42<00:00, 357.15s/it]


## Individual plots

In [17]:
global_results.make_experiment_plots()

## General plot

In [18]:
# import pickle
# with open('data2.db','wb') as outp:
#     pickle.dump(global_results,outp,-1)

In [22]:
global_results.make_global_plot()

AttributeError: 'ExperimentalSetup' object has no attribute 'image_filters'